<a href="https://colab.research.google.com/github/adidror005/youtube-videos/blob/main/MultiLegOptions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install alpaca-py
from google.colab import userdata
from alpaca.trading.client import TradingClient
ALPACA_API_KEY = userdata.get('ALPACA_API_KEY')
ALPACA_API_SECRET = userdata.get('ALPACA_API_SECRET')

trading_client = TradingClient(ALPACA_API_KEY, ALPACA_API_SECRET, paper=True)

### Look at Open Orders

In [4]:
trading_client.get_orders()

[]

# Utility Function to Get Option Symbols

In [13]:
from alpaca.trading.requests import GetOptionContractsRequest
from alpaca.trading.enums import ContractType
def get_option_symbol(symbol, expiry, strike, option_type):
    option_contracts = trading_client.get_option_contracts(
        GetOptionContractsRequest(
            underlying_symbols=[symbol],
            expiration_date=expiry,
            strike_price_gte=str(strike),
            strike_price_lte=str(strike),
            type=option_type
        )
    ).option_contracts

    if len(option_contracts) != 1:
        raise ValueError("Did not find exactly 1 unique contract.")
    return option_contracts[0].symbol
get_option_symbol('AMD','2025-05-30',100,ContractType.CALL),get_option_symbol('NVDA','2025-05-23',120,ContractType.CALL)

('AMD250530C00100000', 'NVDA250523C00120000')

### Define The Legs of the Contract (i.e. for Diagonal Call Spread)
* symbol
* side
* ratio_qty


In [19]:
from alpaca.trading.requests import OptionLegRequest
from alpaca.trading.enums import OrderSide
legs = [
OptionLegRequest(
        symbol='AMD250523C00100000',
        side=OrderSide.BUY,
        ratio_qty=1
    ),
OptionLegRequest(
        symbol='AMD250523C00120000',
        side=OrderSide.SELL,
        ratio_qty=1
    )
]


### Define Order Request and Submit Order
* order_class = OrderClass.MLEG
* legs = list[OptionLegRequest]
* Note: what is cool is we can define our own id via the *client_id* so we can manage IDs ourselves.


In [20]:
from alpaca.trading.requests import LimitOrderRequest, MarketOrderRequest
from alpaca.trading.enums import OrderSide, OrderClass, TimeInForce
from alpaca.trading.client import TradingClient
# Order for the iron condor
req = LimitOrderRequest(
    qty=1,
    order_class=OrderClass.MLEG,
    time_in_force=TimeInForce.DAY,
    legs=legs,
    limit_price=5,  # i.e., for a net price of 0
    client_order_id = 'example50'
)
res = trading_client.submit_order(req)
res

APIError: {"code":42210000,"message":"equity legs are allowed only with a corresponding amount of call contracts"}

### Get Order By Client Id

### Cancel Orders if Necessary

In [ ]:
trading_client.cancel_orders()

[{   'body': None,
     'id': UUID('265ac2ec-e75d-4214-b6dd-f7cb752d0e4c'),
     'status': 200},
 {   'body': None,
     'id': UUID('7910d7db-d4cd-4699-8b11-c2bb58165d52'),
     'status': 200},
 {   'body': None,
     'id': UUID('b5687e32-ec46-476d-b60d-0a773d5fef82'),
     'status': 200}]

In [ ]:
trading_client.get_order_by_client_id('example4')

{   'asset_class': None,
    'asset_id': None,
    'canceled_at': None,
    'client_order_id': 'example4',
    'created_at': datetime.datetime(2025, 5, 20, 17, 43, 46, 421211, tzinfo=TzInfo(UTC)),
    'expired_at': None,
    'expires_at': None,
    'extended_hours': False,
    'failed_at': None,
    'filled_at': None,
    'filled_avg_price': None,
    'filled_qty': '0',
    'hwm': None,
    'id': UUID('70e6f55d-7c9a-4d5b-83ed-d3959595c89c'),
    'legs': [   {   'asset_class': <AssetClass.US_OPTION: 'us_option'>,
                    'asset_id': UUID('4504feaf-117a-479d-b51c-3ff0a539f207'),
                    'canceled_at': None,
                    'client_order_id': 'f63a24e3-c82f-4970-aee1-c12ad81b5083',
                    'created_at': datetime.datetime(2025, 5, 20, 17, 43, 46, 421211, tzinfo=TzInfo(UTC)),
                    'expired_at': None,
                    'expires_at': datetime.datetime(2025, 5, 20, 20, 0, tzinfo=TzInfo(UTC)),
                    'extended_hours': False,


### Replace Order
* Can only replace limit_price, qty--- Not individual symbols or ratios.

In [11]:
from alpaca.trading.requests import ReplaceOrderRequest
res_rep=trading_client.replace_order_by_id(res.id,ReplaceOrderRequest(limit_price=15,client_order_id='rep'))

NameError: name 'res' is not defined

In [12]:
trading_client.get_order_by_client_id('rep')

{   'asset_class': None,
    'asset_id': None,
    'canceled_at': None,
    'client_order_id': 'rep',
    'created_at': datetime.datetime(2025, 5, 20, 17, 52, 46, 565024, tzinfo=TzInfo(UTC)),
    'expired_at': None,
    'expires_at': datetime.datetime(2025, 5, 20, 20, 0, tzinfo=TzInfo(UTC)),
    'extended_hours': False,
    'failed_at': None,
    'filled_at': datetime.datetime(2025, 5, 20, 17, 52, 46, 614911, tzinfo=TzInfo(UTC)),
    'filled_avg_price': '13.26',
    'filled_qty': '1',
    'hwm': None,
    'id': UUID('86d9f171-ba60-45b0-89e6-a2266d2e7029'),
    'legs': [   {   'asset_class': <AssetClass.US_OPTION: 'us_option'>,
                    'asset_id': UUID('4504feaf-117a-479d-b51c-3ff0a539f207'),
                    'canceled_at': None,
                    'client_order_id': '444b6b1c-10f8-4bd1-bf28-e5307c1e00c5',
                    'created_at': datetime.datetime(2025, 5, 20, 17, 52, 46, 565024, tzinfo=TzInfo(UTC)),
                    'expired_at': None,
                    '